# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
  import nltk
  nltk.download(['punkt', 'wordnet','averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
# import libraries
import re
import pandas as pd
import sqlalchemy as db
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# load data from database
engine = db.create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('CategorisedMessages', engine)
X = df['message']
Y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Replace none-character with space
    text = re.sub('[^A-Za-z0-9]',' ',text)
    
    # Tokenize the input
    tokens = word_tokenize(text)
    
    # Initialize lemmatizer for standardize form of words
    lemmatizer = WordNetLemmatizer()
    
    # We will iterate each token in the list, lemmatize and return result
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

In [10]:
y_test.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [12]:
df_pred = pd.DataFrame(y_pred)
df_pred.columns=y_test.columns

for column in y_test:
    print(column)
    print(classification_report(y_test[column],df_pred[column]))

related
             precision    recall  f1-score   support

          0       0.63      0.39      0.48      1487
          1       0.83      0.93      0.88      5020
          2       0.57      0.34      0.43        47

avg / total       0.79      0.80      0.79      6554

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5432
          1       0.80      0.37      0.50      1122

avg / total       0.87      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.70      0.88      0.78      3744
          1       0.77      0.51      0.61      2810

avg / total       0.73      0.72      0.71      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f65f5b59f28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 6. Improve your model
Use grid search to find better parameters. 

In [37]:
parameters = {
     'clf__estimator__n_estimators': [10,20]
}

cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=4)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [35]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

In [36]:
df_pred2 = pd.DataFrame(y_pred)
df_pred2.columns=y_test.columns

for column in y_test:
    print(column)
    print(classification_report(y_test[column],df_pred2[column]))

related
             precision    recall  f1-score   support

          0       0.64      0.38      0.47      1487
          1       0.83      0.93      0.88      5020
          2       0.50      0.45      0.47        47

avg / total       0.79      0.80      0.78      6554

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5432
          1       0.83      0.36      0.50      1122

avg / total       0.87      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.71      0.88      0.78      3744
          1       0.76      0.52      0.61      2810

avg / total       0.73      0.72      0.71      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [37]:
# Function that count number of adv and adj in the sentence and return back as new feature
class AdvAdjExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        token_list = tokenize(text)
        pos_tags = nltk.pos_tag(token_list)
        adj_adv = 0
        for word, tag in pos_tags:
            if tag in ['JJ','JJR','JJS','RB','RBR','RBS']:
                adj_adv = adj_adv+1
        return adj_adv

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [59]:
# Add number of adj and adv count as new feature and try again
pipeline2 = Pipeline([
    ('features',FeatureUnion([
        ('text_pipeline',Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('adv_adj', AdvAdjExtractor())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

parameters2 = {
    'clf__estimator__n_estimators': [10,20]
}

cv2 = GridSearchCV(pipeline2, param_grid=parameters2,n_jobs=10)

In [60]:
cv2.fit(X_train, y_train)
y_pred = cv2.predict(X_test)

In [61]:
# Evaluate improvement result
df_pred3 = pd.DataFrame(y_pred)
df_pred3.columns=y_test.columns

for column in y_test:
    print(column)
    print(classification_report(y_test[column],df_pred3[column]))

related
             precision    recall  f1-score   support

          0       0.67      0.32      0.43      1487
          1       0.82      0.95      0.88      5018
          2       0.46      0.24      0.32        49

avg / total       0.78      0.80      0.78      6554

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.93      5434
          1       0.87      0.38      0.53      1120

avg / total       0.88      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6518
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.74      0.86      0.79      3819
          1       0.75      0.57      0.65      2735

avg / total       0.74      0.74      0.73      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [62]:
pickle.dump(cv2, open('message_classification_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [4]:
import nltk
nltk.download(['punkt', 'wordnet','averaged_perceptron_tagger'])

import sys
import re
import pandas as pd
import sqlalchemy as db
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier

class AdvAdjExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        token_list = tokenize(text)
        pos_tags = nltk.pos_tag(token_list)
        adj_adv = 0
        for word, tag in pos_tags:
            if tag in ['JJ','JJR','JJS','RB','RBR','RBS']:
                adj_adv = adj_adv+1
        return adj_adv

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

def load_data(database_filepath):
    engine = db.create_engine('sqlite:///'+database_filepath)
    df = pd.read_sql_table('CategorisedMessages', engine)
    category_names = ['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']
    X = df['message']
    Y = df[category_names]
    
    return X, Y, category_names


def tokenize(text):
    # Replace none-character with space
    text = re.sub('[^A-Za-z0-9]',' ',text)
    
    # Tokenize the input
    tokens = word_tokenize(text)
    
    # Initialize lemmatizer for standardize form of words
    lemmatizer = WordNetLemmatizer()
    
    # We will iterate each token in the list, lemmatize and return result
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


def build_model():
    # Add number of adj and adv count as new feature and try again
    pipeline = Pipeline([
        ('features',FeatureUnion([
            ('text_pipeline',Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('adv_adj', AdvAdjExtractor())
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    parameters = {
        'clf__estimator__n_estimators': [10,20]
    }

    return GridSearchCV(pipeline, param_grid=parameters,n_jobs=4)
    

def evaluate_model(model, X_test, Y_test, category_names):
    y_pred = model.predict(X_test)
    
    # Convert to dataframe for ease of iteration
    df_y_pred = pd.DataFrame(y_pred)
    df_y_pred.columns=category_names
    
    # Evaluate improvement result
    for column in category_names:
        print(column)
        print(classification_report(Y_test[column],df_y_pred[column]))


def save_model(model, model_filepath):
    pickle.dump(model, open(model_filepath, 'wb'))


def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Loading data...
    DATABASE: InsertDatabaseName.db
Building model...
Training model...
Evaluating model...
related
             precision    recall  f1-score   support

          0       0.68      0.32      0.43      1237
          1       0.81      0.95      0.88      3970
          2       0.62      0.28      0.38        36

avg / total       0.78      0.80      0.77      5243

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      4355
          1       0.87      0.43      0.57       888

avg / total       0.89      0.89 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Trained model saved!
